In [19]:
! pip install -r requirements.txt

  Using cached streamlit-1.50.0-py3-none-any.whl.metadata (9.5 kB)
ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5
ERROR: Could not find a version that satisfies the requirement shapely.geometry (from versions: none)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
ERROR: No matching distribution found for shapely.geometry


In [20]:
! pip install shapely psycopg2 geopandas folium tqdm sqlalchemy


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [21]:
import functions

ModuleNotFoundError: No module named 'google'